In [77]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic


In [38]:
src_path = r"./ET_RCS_28SEP15_quora_data_emerging_technologies.csv" 
df = pd.read_csv(src_path)

In [39]:
df

,Unnamed: 0,Answer,followerCount,name,upvoteCount,answerCount,answerViews,question
0,0,50 years is aLong distance travel will be by h...,"45,199 followers",David Seidman,999,3.6K,82.9M,https://www.quora.com/What-will-the-worlds-tec...
1,1,Faster elevators capable of moving sideways.Fa...,"4,975 followers",Bobby Tatro,3.9K,2.1K,44.7M,https://www.quora.com/What-will-the-worlds-tec...
2,2,the only answer I can give you is that in that...,2 followers,Phillip Jensen,9,0,0,https://www.quora.com/What-will-the-worlds-tec...
3,3,"Well, the first question will be:How will glob...",1 follower,Frank Robertson,9,585,1.2M,https://www.quora.com/What-will-the-worlds-tec...
4,4,Go back to 1971 and describe in detail present...,"2,454 followers",Edward Lafreniere,99,12.2K,2M,https://www.quora.com/What-will-the-worlds-tec...
...,...,...,...,...,...,...,...,...
634,634,2020 will leave its mark in history as the yea...,1 follower,Debabrata,9,0,0,https://www.quora.com/What-are-the-solutions-t...
635,635,The biggest problem I see with communication i...,UNKNOWN,MaxEd,9,0,0,https://www.quora.com/What-are-the-solutions-t...
636,636,"Over the years, technology was being infused i...",651 followers,Almarie Meyer,9,1K,955.5K,https://www.quora.com/What-are-the-solutions-t...
637,637,Universities that apply technology in all its ...,0 followers,Hritvik Gaur,9,219,315.9K,https://www.quora.com/What-are-the-solutions-t...


In [40]:
#remove nan rows in Answer column
df = df[df['Answer'].isnull() ==False]

In [41]:
df

,Unnamed: 0,Answer,followerCount,name,upvoteCount,answerCount,answerViews,question
0,0,50 years is aLong distance travel will be by h...,"45,199 followers",David Seidman,999,3.6K,82.9M,https://www.quora.com/What-will-the-worlds-tec...
1,1,Faster elevators capable of moving sideways.Fa...,"4,975 followers",Bobby Tatro,3.9K,2.1K,44.7M,https://www.quora.com/What-will-the-worlds-tec...
2,2,the only answer I can give you is that in that...,2 followers,Phillip Jensen,9,0,0,https://www.quora.com/What-will-the-worlds-tec...
3,3,"Well, the first question will be:How will glob...",1 follower,Frank Robertson,9,585,1.2M,https://www.quora.com/What-will-the-worlds-tec...
4,4,Go back to 1971 and describe in detail present...,"2,454 followers",Edward Lafreniere,99,12.2K,2M,https://www.quora.com/What-will-the-worlds-tec...
...,...,...,...,...,...,...,...,...
633,633,Thank you for A2A.Ethiopia is the fastest-grow...,0 followers,David Harden,9,246,48.6K,https://www.quora.com/What-are-the-solutions-t...
634,634,2020 will leave its mark in history as the yea...,1 follower,Debabrata,9,0,0,https://www.quora.com/What-are-the-solutions-t...
635,635,The biggest problem I see with communication i...,UNKNOWN,MaxEd,9,0,0,https://www.quora.com/What-are-the-solutions-t...
636,636,"Over the years, technology was being infused i...",651 followers,Almarie Meyer,9,1K,955.5K,https://www.quora.com/What-are-the-solutions-t...


In [17]:
ans_text = df['Answer']

In [26]:
ans_text = ans_text.to_list()

In [42]:
#create topic model without stop words
vectorizer_model = CountVectorizer(stop_words="english")

topic_model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(ans_text)



In [43]:
topic_model.get_topic_info() 

,Topic,Count,Name
0,-1,134,-1_technology_cryptocurrency_using_like
1,0,80,0_3d_technologies_technology_cars
2,1,55,1_fusion_energy_nuclear_plasma
3,2,53,2_brain_lobe_body_jess
4,3,42,3_50_years_world_people
5,4,30,4_iot_internet_things_devices
6,5,25,5_computers_years_thisor_desktops
7,6,25,6_following_reasonable_2031_2068
8,7,25,7_technology_human_google_use
9,8,24,8_technology_students_communication_issue


In [56]:
topic_model.get_topic(0)

[('3d', 0.038612547890924956),
 ('technologies', 0.036247617208764224),
 ('technology', 0.02685682304835644),
 ('cars', 0.025960401845572707),
 ('printing', 0.025184528189677376),
 ('emerging', 0.02514236854565453),
 ('10', 0.02351136559128775),
 ('predictions', 0.020639688773241776),
 ('printers', 0.019905773507859565),
 ('future', 0.01926768007074218)]

In [73]:
#The top n terms per topic and their respective c-TF-IDF values
topic_rep_dict = topic_model.topic_representations_

topic_rep_labels = topic_model.topic_labels_
topic_rep_docs = topic_model.representative_docs_

In [70]:
topic_rep_labels

{-1: '-1_technology_cryptocurrency_using_like',
 0: '0_3d_technologies_technology_cars',
 1: '1_fusion_energy_nuclear_plasma',
 2: '2_brain_lobe_body_jess',
 3: '3_50_years_world_people',
 4: '4_iot_internet_things_devices',
 5: '5_computers_years_thisor_desktops',
 6: '6_following_reasonable_2031_2068',
 7: '7_technology_human_google_use',
 8: '8_technology_students_communication_issue',
 9: '9_union_economy_china_soviet',
 10: '10_intelligence_ai_programming_cars',
 11: '11_solar_meat_cooking_food',
 12: '12_kaspersky_future_predict_machines',
 13: '13_start_remember_popular_write',
 14: '14_biological_capabilities_human_grow',
 15: '15_civil_door_open_just',
 16: '16_finding_jobs_development_longterm'}

In [94]:
#save dicts

with open('topic_representations', 'w') as f:
    json.dump(topic_rep_dict, f)


with open('topic_rep_labels', 'w') as f:
    json.dump(topic_rep_labels, f)
 

In [96]:
df_rep_docs = pd.DataFrame(topic_rep_docs).T

In [97]:
df_rep_docs.to_csv('representative_sentences_per_topic')